## Indice
#### 1. Pulizia crawled data
#### 2.Creo una colonna con i tag puliti
#### 3.Creazione network di partenza
###### 4.1 Controllo preliminare dei nodi ed edges
#### 5. Risultati

In [1]:
import twint
import pandas as pd


import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
warnings.filterwarnings('ignore')


%matplotlib inline 

### PULIZIA CRAWLED DATA

In [2]:
df = pd.read_csv("../Dataset FFF/climatechange.csv", sep='\t')

In [3]:
#riduco le dimensioni del dataframe per avvicinarcio ai 15k, lo faccio togliendo parte finale dei dati 
df = df.iloc[:41700]

In [4]:
df.tail(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
41698,1187693451556610049,1187693451556610049,2019-10-25 13:32:39 ora legale Europa occidentale,2019-10-25,13:32:39,200,712989553,just_aenna,Anna Hübner,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41699,1187693166742429696,1187693166742429696,2019-10-25 13:31:31 ora legale Europa occidentale,2019-10-25,13:31:31,200,68459659,winterbe_,Benjamin,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [5]:
# Get all names 
for col_name in df.columns: 
    print(col_name)

id
conversation_id
created_at
date
time
timezone
user_id
username
name
place
tweet
language
mentions
urls
photos
replies_count
retweets_count
likes_count
hashtags
cashtags
link
retweet
quote_url
video
thumbnail
near
geo
source
user_rt_id
user_rt
retweet_id
reply_to
retweet_date
translate
trans_src
trans_dest


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               41700 non-null  int64  
 1   conversation_id  41700 non-null  int64  
 2   created_at       41700 non-null  object 
 3   date             41700 non-null  object 
 4   time             41700 non-null  object 
 5   timezone         41700 non-null  int64  
 6   user_id          41700 non-null  int64  
 7   username         41700 non-null  object 
 8   name             41697 non-null  object 
 9   place            12 non-null     object 
 10  tweet            41700 non-null  object 
 11  language         41700 non-null  object 
 12  mentions         41700 non-null  object 
 13  urls             41700 non-null  object 
 14  photos           41700 non-null  object 
 15  replies_count    41700 non-null  int64  
 16  retweets_count   41700 non-null  int64  
 17  likes_count 

Toglie created_at, time, timezone, lasciando la data
Sono dati ripetuti 

In [7]:
df.drop(['trans_dest', 'trans_src','translate','retweet_date','geo','source','user_rt_id', 'user_rt','retweet_id',
        'near','place','thumbnail', 'time', 'timezone', 'video', 'photos', 'cashtags', 'created_at','quote_url','id','conversation_id'], axis=1, inplace=True)

In [8]:
#No valori nulli!
df['language'].isnull().values.any()

False

In [9]:
Lingue=list(set(df['language']))
#Lingue

Restingendo il range di tweet con numero di likes, retweet e replies
Si arriva a 16335 valori totali

In [10]:
df=df[df['likes_count']>=10]
df=df[df['replies_count']>=3]
df=df[df['retweets_count']>=3]

In [11]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16335 entries, 6 to 41699
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            16335 non-null  object
 1   user_id         16335 non-null  int64 
 2   username        16335 non-null  object
 3   name            16334 non-null  object
 4   tweet           16335 non-null  object
 5   language        16335 non-null  object
 6   mentions        16335 non-null  object
 7   urls            16335 non-null  object
 8   replies_count   16335 non-null  int64 
 9   retweets_count  16335 non-null  int64 
 10  likes_count     16335 non-null  int64 
 11  hashtags        16335 non-null  object
 12  link            16335 non-null  object
 13  retweet         16335 non-null  bool  
 14  reply_to        16335 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 1.9+ MB


### Creo una colonna con i tag puliti

In [12]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Faccio questa funzione per pulire i tags dalle quadre, la applico a una nuova colonna (clean_tags)

In [13]:
def remove_content(text):
    text = re.sub(r"\[", "", text) #remove ]
    text=re.sub(r'\]','',text) #remove ]
    text=re.sub(r'\'','',text) #remove apici
    text=re.sub(r'\n','',text) #remove spazio vuoto

    return text

In [14]:
len(df['hashtags'])

16335

In [15]:
df['clean_tags']=df['hashtags'].apply(lambda x: remove_content(x))
len(df['clean_tags'])

16335

In [16]:
df.insert(0, 'clean_tags', df.pop('clean_tags'))

In [17]:
df.insert(1, 'hashtags', df.pop('hashtags'))

In [18]:
df.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
6,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]


In [19]:
#Salvo le modifiche nel df nuovo
df.to_csv('../Dataset FFF/fff.csv', index=False)

###  Creazione network di partenza

In [20]:
df1=pd.read_csv("../Dataset FFF/fff.csv")
df1.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]


In [21]:
#Ti crea un dizionario in cui le chiavi sono i tag singoli e i valori sono gli id degli user
tags_dict = {}
for i in range(len(df1)):
    ll = str(df1["clean_tags"][i]).split(",")[:-1]
    for tg in ll:
        if tg not in tags_dict:
            tags_dict[tg] = [df1["tweet"][i]]
        else:
            tags_dict[tg].append(df1["tweet"][i])

In [22]:
#Crea la lista di tags dalle chiavi del dizionario e li associa alla variabile tag
tags = list(tags_dict.keys())
#crea una lista costituita da coppie di tags
couple_tags = []
for i in range(len(tags)):
    for j in tags[i+1:]:
        couple_tags.append((tags[i], j)) 

In [23]:
#couple_tags

In [24]:
len(couple_tags)

18437628

In [25]:
len(tags)

6073

In [26]:
source = []
target = []
count = []
nodes = []
for i in couple_tags:
    ll = list(set(tags_dict[i[0]]) & set(tags_dict[i[1]]))
    if len(ll)>0:
        source.append(i[0])
        target.append(i[1])
        count.append(len(ll))
        nodes.append(i[0])
        nodes.append(i[1])

In [27]:
nodes = list(set(nodes)) #list()crea una lista; set(nodes) prende i nodi senza ripetizioni

In [28]:
len(nodes)

5649

In [29]:
nodes

[' colombiasos',
 ' ldk22nrw',
 ' standwithsapmi',
 ' mittelstand',
 'demospruch',
 ' disruption',
 ' dogs',
 'photography',
 ' cop26',
 'politprofis',
 ' guerillapv',
 ' python',
 ' карпати',
 'wasserstoff',
 '49th',
 ' nachhaltigkeit',
 ' globe',
 ' bundjugend',
 'volimproklima',
 ' environment',
 ' oligarchs',
 ' union',
 ' digitalindia',
 ' plasticfree',
 ' uganda',
 ' racialjustice',
 ' hannover',
 'extinctionrebellion',
 ' mehrheit',
 '100ee30',
 ' gruenejugend',
 'hamburgwähltklima',
 ' klimatet',
 'nuclear',
 ' 1for25',
 ' nordstream2',
 ' ardwahlarena',
 ' coronavirusde',
 ' stopputinnow',
 ' klimawahl2021',
 ' world',
 ' sek',
 ' mbpoli',
 ' steinmeier',
 ' wirtschaftsministerium',
 'fossil',
 'klimakatastrophe',
 ' petrolio',
 ' realzero',
 'meintwitterjubiläum',
 ' frühstart',
 ' standwithukraine',
 ' renewableenergy',
 'digitalstrike',
 ' photographer',
 'nürnberg',
 'luisa',
 ' bund',
 ' hamas',
 'conservation',
 'kalifornien',
 ' ambientemw',
 ' keingradweiter',
 'kobe',

In [30]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [31]:
new_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
new_df.to_csv("../Dataset FFF/edges.csv", index=False)

In [32]:
others_df = pd.DataFrame({"Id": nodes})
others_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [33]:
others_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
others_df.to_csv("../Dataset FFF/edges.csv", index=False)

### Creazione dataset da 10k e 3k nodi

###### Controllo preliminare dei nodi ed edges

qui però stiamo selezionando i primi 10k e 3k quindi stiamo proprio tagliando il dataset, questo potrebbe essere un problema dato che potrebbe non essere rappresentativo di tutto il dataset


In [34]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math
import numpy as np
import networkx as nx
from operator import itemgetter

In [35]:
path = "../Dataset FFF/edges.csv"
path_all = "../Dataset FFF/fff.csv"
path_nodes = "../Dataset FFF/nodes.csv"

In [36]:
fff = pd.read_csv(path)

In [37]:
fff.sort_values('Count', ascending=False)

,Source,Target,Count
2736,fridaysforfuture,climatestrike,554
9173,therapeutenamlimit,heimatstern,551
9197,einesorgeweniger,heimatstern,528
9228,heimatstern,ohnemeinenphysiotherapeuten,525
9172,therapeutenamlimit,einesorgeweniger,524
...,...,...,...
11673,klimawandel,elektroautos,1
11672,klimawandel,klimakatastrophe,1
11670,klimawandel,parentsforfuture,1
11669,klimawandel,langstreckenluisa,1


In [38]:
G = nx.read_edgelist(path, comments = "S", delimiter = ",", data=(("Count", int),))

In [39]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print(" ---- Top 20 nodes by degree ----")
for d in sorted_degree[:20]:
    print("\t" + str(d))

 ---- Top 20 nodes by degree ----
	('fridaysforfuture', 3864)
	('climatestrike', 1015)
	('climatecrisis', 671)
	('climateaction', 658)
	('climateemergency', 489)
	('climatechange', 444)
	('klimaschutz', 398)
	('climatestrikeonline', 358)
	('klimakrise', 338)
	('climate', 291)
	('klimastreik', 276)
	('climateactionnow', 273)
	('climatejustice', 271)
	('schoolstrike4climate', 260)
	('gretathunberg', 254)
	('klimawandel', 247)
	('fff', 241)
	('fridaymotivation', 219)
	('fridayfeeling', 212)
	('facetheclimateemergency', 201)


In [40]:
nodes = []
others = []
for i in range(len(sorted_degree)):
    if i<10000:
        nodes.append(sorted_degree[i][0])
        #print("N.nodi:",nodes)
for i in range(len(sorted_degree)):
    if i<3000:
        others.append(sorted_degree[i][0])

##### Abbiamo 15k nodi

In [41]:
len(nodes)

7216

In [42]:
new_fff = fff[(fff["Source"].isin(nodes)) & (fff["Target"].isin(nodes))] 

In [43]:
new_df.to_csv("../Dataset FFF/edges_10K.csv",  index=False)

In [44]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes_10K.csv", index=False)

In [45]:
edges=pd.read_csv('../Dataset FFF/edges_10k.csv')

## Risultato pulizia:

In [46]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 7216
Number of edges: 28103
Average degree:   7.7891


In [47]:
new_df

,Id
0,fridaysforfuture
1,climatestrike
2,climatecrisis
3,climateaction
4,climateemergency
...,...
7211,diwali
7212,energieeffizienz
7213,ieselparador
7214,python
